In [ ]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:

train = pd.read_csv('//Users/paul/Sites/kaggle_competitions/titanic/data/train.csv')
train['Age'].fillna(train['Age'].mean(), inplace=True)

def hasAlias(name):
    return True if "(" in name else False

def getTitle(name):
    return name[name.find(",")+1:name.find(".")]

train['Title'] = train.Name.apply(getTitle)
train['Deck'] = train['Cabin'].astype(str).str[0]
train['HasAlias'] = train.Name.apply(hasAlias)

countVectorizerMap =[
    {'name':'Pclass', 'analyzer':'char'},
    {'name': 'Sex', 'analyzer': 'word'},
    {'name': 'Parch', 'analyzer': 'char'},
    {'name': 'Embarked', 'analyzer': 'char'},
    {'name': 'HasAlias', 'analyzer': 'word'},
    {'name': 'Title', 'analyzer': 'word'},
    {'name': 'Deck', 'analyzer': 'char'}
]


def _fitTransform(column):
    vectorizer = CountVectorizer(analyzer=column['analyzer'], lowercase=False)
    matrix = vectorizer.fit_transform(train[column['name']].apply(str)).todense()
    columnNames = [column['name'] + "_"+ x for x in vectorizer.vocabulary_]
    
    return [ pd.DataFrame(data=matrix, columns=columnNames), vectorizer ]


fields = [_fitTransform(x) for x in countVectorizerMap]
categoryFeatures, vectorizers = zip(*fields)

In [ ]:
categoryFeaturesDF = pd.concat(categoryFeatures, axis=1, join="outer")

In [ ]:
tf.convert_to_tensor(categoryFeaturesDF['Pclass_3'].values, dtype=np.int32)

In [ ]:
{x: tf.convert_to_tensor(categoryFeaturesDF[x].values, dtype=np.int32) for x in  categoryFeaturesDF.columns}

In [ ]:


x = np.hstack([train.as_matrix(["Age", "SibSp", "Fare"]), np.hstack(list(categoryFeatures))])
y = train['Survived'].values

In [ ]:
x.shape

In [ ]:
X = tf.convert_to_tensor(x)
Y = tf.convert_to_tensor(y)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.batch(32)

In [ ]:
(a, b ) = tfe.Iterator(dataset).next()

In [ ]:
a

In [ ]:
dataset

In [ ]:
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

def grad(model, inputs, targets):
  with tfe.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

In [ ]:
dataset = tf.data.Dataset.from_tensors((X, Y))
# dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(32)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(70, activation="relu", input_shape=(49,)),
    tf.keras.layers.Dense(35, activation="relu"),
    tf.keras.layers.Dense(2)
])

train_loss_results = []
train_accuracy_results = []

num_epochs = 300

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()

    # Training loop - using batches of 32
    for (x, y) in tfe.Iterator(dataset):
        # Optimize the model
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables),
                                global_step=tf.train.get_or_create_global_step())

        # Track progress
        epoch_loss_avg(loss(model, x, y))  # add current batch loss
        # compare predicted label to actual label
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                    epoch_loss_avg.result(),
                                                                    epoch_accuracy.result()))